In [1]:
import pandas as pd
import numpy as np
import wbgapi as wb
import re
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV,StratifiedKFold,train_test_split,cross_val_score
from sklearn import metrics
import pickle

In [2]:
final_df=pd.read_csv('WorldBank_Data.csv')

In [3]:
final_df=final_df.loc[:,['Fixed telephone subscriptions','Urban population','Rural population','Population ages 65 and above, male','Population ages 65 and above, female', 'Population ages 15-64, female','Population ages 15-64, male', 'Population ages 0-14, male', 'Population ages 0-14, female','Agriculture, forestry, and fishing, value added (current US$)','Industry (including construction), value added (current US$)','Manufacturing, value added (current US$)','Services, value added (current US$)','Mobile cellular subscriptions','Fixed broadband subscriptions','Air transport, registered carrier departures worldwide']]

In [4]:
final_df=final_df.loc[(final_df['Air transport, registered carrier departures worldwide']>0),]

In [5]:
print(final_df['Air transport, registered carrier departures worldwide'].skew())
print(np.log(final_df['Air transport, registered carrier departures worldwide']+1).skew())

11.38010706955218
0.18108632636786198


In [6]:
# Transforming the target variable to make it a more normal distributiob
final_df['Ln_Y']=np.log(final_df['Air transport, registered carrier departures worldwide']+1)

In [7]:
final_df.shape

(7443, 17)

#### Feature Engineering

In [8]:
x_features=['Fixed telephone subscriptions','Urban population', 'Rural population','Population ages 65 and above, male','Population ages 65 and above, female', 'Population ages 15-64, female','Population ages 15-64, male', 'Population ages 0-14, male', 'Population ages 0-14, female','Agriculture, forestry, and fishing, value added (current US$)','Industry (including construction), value added (current US$)','Manufacturing, value added (current US$)','Services, value added (current US$)','Mobile cellular subscriptions','Fixed broadband subscriptions']

In [9]:
y_feature='Ln_Y'

In [10]:
# Feature Engineering - Selecting Top 4 Features
xgb_fe_mod=XGBRegressor(random_state=12345)
xgb_fe_mod.fit(final_df[x_features],final_df[y_feature])
fe_df=pd.concat([pd.Series(x_features),pd.Series(xgb_fe_mod.feature_importances_)],axis=1)
fe_df.columns=['Features','Importance']
print(fe_df.sort_values('Importance',ascending=False))

                                             Features  Importance
0                       Fixed telephone subscriptions    0.417255
12                Services, value added (current US$)    0.096480
11           Manufacturing, value added (current US$)    0.071058
3                  Population ages 65 and above, male    0.050417
10  Industry (including construction), value added...    0.043101
13                      Mobile cellular subscriptions    0.042799
4                Population ages 65 and above, female    0.042454
2                                    Rural population    0.037112
14                      Fixed broadband subscriptions    0.035821
7                          Population ages 0-14, male    0.032120
6                         Population ages 15-64, male    0.030751
5                       Population ages 15-64, female    0.027554
1                                    Urban population    0.026896
8                        Population ages 0-14, female    0.025543
9   Agricu

In [25]:
features_list=list(fe_df.sort_values('Importance',ascending=False).head(3)['Features'])
features_list

['Fixed telephone subscriptions',
 'Services, value added (current US$)',
 'Manufacturing, value added (current US$)']

#### Train Test Split

In [26]:
train_X,test_X,train_Y,test_Y=train_test_split(final_df[features_list],final_df[['Ln_Y','Air transport, registered carrier departures worldwide']],test_size=0.3,random_state=42)

#### Model Building

In [27]:
# XGBoost
xgb_param_list=[{'n_estimators':[10,30,50],'subsample':[0.8],'learning_rate':[0.001,0.01,0.1]}]
xgb_mod_cv=GridSearchCV(XGBRegressor(random_state=38),xgb_param_list,cv=5,verbose=0,n_jobs=-1,scoring='neg_mean_squared_error')
xgb_mod_cv.fit(train_X,train_Y['Ln_Y'])
xgb_mod=xgb_mod_cv.best_estimator_

In [28]:
print('MAPE: ',metrics.mean_absolute_percentage_error(np.exp(xgb_mod.predict(train_X)+1),train_Y['Air transport, registered carrier departures worldwide']))
print('MAPE: ',metrics.mean_absolute_percentage_error(np.exp(xgb_mod.predict(test_X)+1),test_Y['Air transport, registered carrier departures worldwide']))

MAPE:  0.5805194839173112
MAPE:  0.5860357290690721


In [29]:
# CatBoost
cb_mod=CatBoostRegressor(random_state=12345,verbose=0)
cb_mod.fit(train_X,train_Y['Ln_Y'],eval_set=(test_X,test_Y['Ln_Y']),early_stopping_rounds=10)

In [30]:
print('MAPE: ',metrics.mean_absolute_percentage_error(np.exp(cb_mod.predict(train_X)+1),train_Y['Air transport, registered carrier departures worldwide']))
print('MAPE: ',metrics.mean_absolute_percentage_error(np.exp(cb_mod.predict(test_X)+1),test_Y['Air transport, registered carrier departures worldwide']))

MAPE:  0.598505595388965
MAPE:  0.6080676100151912


In [31]:
# LightGBM
X_train_cp = train_X.copy()
X_train_cp = X_train_cp.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
lgb_param_list=[{'n_estimators':[10,30,50],'subsample':[0.8],'learning_rate':[0.001,0.01,0.1]}]
lgb_mod_cv=GridSearchCV(LGBMRegressor(random_state=38),lgb_param_list,cv=5,verbose=0,n_jobs=-1,scoring='neg_mean_squared_error')
lgb_mod_cv.fit(X_train_cp,train_Y['Ln_Y'])
lgb_mod=lgb_mod_cv.best_estimator_

In [32]:
X_test_cp = test_X.copy()
X_test_cp = X_test_cp.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
print('MAPE: ',metrics.mean_absolute_percentage_error(np.exp(lgb_mod.predict(X_train_cp)+1),train_Y['Air transport, registered carrier departures worldwide']))
print('MAPE: ',metrics.mean_absolute_percentage_error(np.exp(lgb_mod.predict(X_test_cp)+1),test_Y['Air transport, registered carrier departures worldwide']))

MAPE:  0.5913122651070295
MAPE:  0.6005407254480666


#### Saving the model

In [38]:
# Saving the machine learning model as pickle file
pickle.dump(xgb_mod, open('final_model_rr.pkl', 'wb'))
# xgb_mod = pickle.load(open('final_model_rr', 'rb'))